In [3]:
# cards.py
class Card:
    def __init__(self, color, value):
        self.color = color
        self.value = value
        self.cardId = None

    def __str__(self):
        return f"{self.color} {self.value}"

    def matches(self, other):
        return self.color == other.color or self.value == other.value or other.color == "Quantum" or self.color == "Quantum"

    def play(self, game):
        # Default card: no special effect, just advance turn
        game.discard_pile.append(self)

In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def card_to_index(card: Card, Hand: list[Card]) -> int:
    for i, c in enumerate(Hand):
        print(f"Checking card at index {i}: {c}")
        if c.color == card.color and c.value == card.value:
            print(f"Returning index: {i}")
            return i
    print("Returning index: -1")
    return -1

def index_to_binary(index: int, num_qubits: int) -> str:
    binary = format(index, f"0{num_qubits}b")
    print(f"Returning binary representation: {binary}")
    return binary

def grover_oracle(target: str) -> QuantumCircuit:
    n = len(target)
    oracle = QuantumCircuit(n)
    for i, bit in enumerate(target):
        if bit == '0':
            oracle.x(i)
    oracle.h(n-1)
    oracle.mcx(list(range(n-1)), n-1)
    oracle.h(n-1)
    for i, bit in enumerate(target):
        if bit == '0':
            oracle.x(i)
    oracle.name = "Oracle"
    print(f"Returning oracle circuit: {oracle}")
    return oracle

def diffusion_operator(n: int) -> QuantumCircuit:
    diff = QuantumCircuit(n)
    diff.h(range(n))
    diff.x(range(n))
    diff.h(n-1)
    diff.mcx(list(range(n-1)), n-1)
    diff.h(n-1)
    diff.x(range(n))
    diff.h(range(n))
    diff.name = "Diffusion"
    print(f"Returning diffusion operator circuit: {diff}")
    return diff

def grover_card_search(player_hand: list[Card], target_card: Card, verbose=True) -> bool:
    num_cards = len(player_hand)
    if num_cards == 0:
        print("Returning False: player hand is empty.")
        return False
    num_qubits = (num_cards - 1).bit_length()

    card_index = card_to_index(target_card, player_hand)
    if card_index == -1:
        if verbose:
            print(f"The card {target_card} is not in the Hand.")
        print("Returning False: card not found in hand.")
        return False

    target_bin = index_to_binary(card_index, num_qubits)
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    qc.append(grover_oracle(target_bin), range(num_qubits))
    qc.append(diffusion_operator(num_qubits), range(num_qubits))
    qc.measure(range(num_qubits), range(num_qubits))

    backend = AerSimulator()
    transpiled = transpile(qc, backend=backend)
    result = backend.run(transpiled, shots=1024).result()
    counts = result.get_counts()
    corrected_counts = {k[::-1]: v for k, v in counts.items()}
    top_result = max(corrected_counts, key=corrected_counts.get)

    if verbose:
        print(f"\nGrover result (corrected): {corrected_counts}")
        print(f"Top result (binary): {top_result} -> index {int(top_result, 2)}")

    result = int(top_result, 2) == card_index
    print(f"Returning result: {result}")
    return result

In [7]:

from qiskit_aer import AerSimulator
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from scipy.optimize import minimize
from qiskit_optimization.translators import to_ising
import numpy as np


class quantum_balance_card(Card):
    def __init__(self, color="Purple"):
        super().__init__(color, "Quantum Balance")
        self.cardId = 10

    def card_weight(self, card):
        if isinstance(card.value, int):
            weight = card.value
        elif card.value == 'Skip':
            weight = 10
        elif card.value == 'Reverse':
            weight = 10
        elif card.value == 'Draw Two':
            weight = 15
        elif card.value == 'Quantum':
            weight = 20
        elif card.value == "DUMMY":
            weight = 0  # Dummy zero weight
        else:
            weight = 5
        print(f"Card weight for {card}: {weight}")
        return weight

    def play(self, game):
        players = [game.get_current_player(), game.get_next_player()]
        all_cards = players[0].Hand + players[1].Hand

        dummy_used = False
        dummy_index = -1
        if len(all_cards) % 2 != 0:
            dummy_card = Card("Dummy", "DUMMY")
            all_cards.append(dummy_card)
            dummy_used = True
            dummy_index = len(all_cards) - 1

        n_cards = len(all_cards)

        # Build QUBO model where each variable x_i indicates which player owns card i:
        # x_i = 0 -> player 0, x_i = 1 -> player 1

        mdl = Model(name="Card_Player_Assignment")
        x = [mdl.binary_var(name=f"x_{i}") for i in range(n_cards)]

        weights = [self.card_weight(card) for card in all_cards]
        total_weight = sum(weights)
        avg_weight = total_weight / 2

        # Penalize difference in total weights:
        weight_diff = mdl.sum(weights[i] * x[i] for i in range(n_cards)) - avg_weight
        obj_weight = weight_diff * weight_diff

        # Penalize difference in number of cards assigned to each player:
        card_diff = mdl.sum(x) - n_cards / 2
        obj_card = card_diff * card_diff

        # Combine objectives with weighting factor alpha for card count balance
        alpha = 40
        mdl.minimize(obj_weight + alpha * obj_card)

        # Convert model to QUBO
        qp = from_docplex_mp(mdl)
        qubo = QuadraticProgramToQubo().convert(qp)

        # Convert to Ising Hamiltonian for quantum solver
        hamiltonian, _ = to_ising(qubo)

        backend = AerSimulator()
        estimator = Estimator(mode=backend)
        pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

        def cost_function(params, estimator, circuit, hamiltonian):
            # Run pass manager once
            isa_psi = pm.run(circuit)
            isa_observables = hamiltonian.apply_layout(isa_psi.layout)
            job = estimator.run([(isa_psi, isa_observables, params)])
            cost = job.result()[0].data.evs
            print(f"Cost function value: {cost}")
            return cost

        circuit_qaoa = QAOAAnsatz(hamiltonian, reps=20)
        p = circuit_qaoa.num_parameters // 2
        gamma_init = np.linspace(0.1, 1.5, p)
        beta_init = np.linspace(0.1, 1.5, p)
        params_init = np.concatenate([gamma_init, beta_init])

        res_opt = minimize(
            cost_function,
            params_init,
            args=(estimator, circuit_qaoa, hamiltonian),
            method="COBYLA"
        )

        params_opt = res_opt.x
        print(f"Optimized parameters: {params_opt}")

        sampler = Sampler(mode=backend)
        circuit_qaoa_copy = circuit_qaoa.decompose(reps=2).copy()
        circuit_qaoa_copy.measure_all()
        counts = sampler.run([(circuit_qaoa_copy, params_opt)]).result()[0].data.meas.get_counts()
        print(f"Sampler counts: {counts}")

        # If counts is dict, get most likely bitstring, else fallback
        if isinstance(counts, dict):
            most_likely_bitstring = max(counts, key=counts.get)
        else:
            # fallback (depends on sampler implementation)
            most_likely_bitstring = None
            for bitstring in counts:
                most_likely_bitstring = bitstring
                break

        if most_likely_bitstring is None:
            raise RuntimeError("Failed to get a bitstring from sampler result.")

        print(f"Most likely bitstring: {most_likely_bitstring}")

        # Assign cards to players based on bitstring (reverse to match indexing)
        new_hands = {0: [], 1: []}
        for i in range(n_cards):
            if dummy_used and i == dummy_index:
                continue
            bit_val = int(most_likely_bitstring[::-1][i])
            new_hands[bit_val].append(all_cards[i])

        # Replace player hands
        for j, player in enumerate(players):
            player.Hand = new_hands[j]
        print(f"New hands assigned: {new_hands}")

In [8]:
# quantum_card.py
# Example of a quantum card that uses Qiskit to create a quantum effect #
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class quantum_card(Card):
    def __init__(self, color, value):
        super().__init__(color, value)
        self.circuit = self.create_circuit()
        self.cardId = 11  # Unique identifier for this card type
    
    def create_circuit(self):
        qc = QuantumCircuit(1)
        qc.h(0)  # Put qubit in superposition
        qc.measure_all()
        print(f"Created quantum circuit: {qc}")
        return qc
    
    def run_quantum_effect(self):
        backend = AerSimulator()
        qc = self.create_circuit()
        transpiled = transpile(qc, backend=backend)
        result = backend.run(transpiled, shots=1).result()
        counts = result.get_counts()
        print(f"Quantum effect result counts: {counts}")
        return counts
    
    def play(self, game):
        super().play(game)
        counts = self.run_quantum_effect()
        print(f"Counts from quantum effect: {counts}")
        # Example effect: if measured '0', skip next player; else reverse
        if '0' in counts:
            game.skip_next_player = True
            print("Effect: Skipping next player.")
        else:
            game.reverse_turn_order()
            print("Effect: Reversing turn order.")

In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

class Quantum_color_card(Card):
    """
    Uses a full 1D discrete-time coined quantum walk to choose a color.
    """

    def __init__(self, color_list, current_color="Quantum"):
        self.colors = color_list
        self.current_color = current_color
        self.cardId = 12
        super().__init__(current_color, "Color")

    def play(self, game):
        self.current_color = game.get_top_card().color
        print(f"Current color set to: {self.current_color}")
        new_color = self.activate_quantum_walk()
        print(f"New color chosen by quantum walk: {new_color}")
        game.current_color = new_color
        self.color = new_color

    def activate_quantum_walk(self, steps=np.random.randint(1, 5)):
        """
        Perform a 1D discrete-time quantum walk with 3 steps on 4 colors.
        Position: 2 qubits (q0 = LSB, q1 = MSB), Coin: 1 qubit (q2)
        """
        from qiskit import QuantumCircuit, transpile
        from qiskit_aer import AerSimulator

        color_map = {
            0: "Red",
            1: "Blue",
            2: "Green",
            3: "Yellow",
        }

        qc = QuantumCircuit(3, 2)  # q0, q1 = position; q2 = coin

        # Initialize walker to current position
        pos_index = self.colors.index(self.current_color)
        print(f"Initial position index: {pos_index}")
        if pos_index & 0b01:
            qc.x(0)  # LSB
        if pos_index & 0b10:
            qc.x(1)  # MSB

        # Initialize coin qubit in |+⟩
        qc.h(2)

        for _ in range(steps):
            # Step 1: Coin toss (Hadamard on coin qubit)
            qc.h(2)

            # Step 2: Shift operator — controlled shift depending on coin
            # If coin == |0>: move left (decrement mod 4)
            # If coin == |1>: move right (increment mod 4)

            # Right shift (|i+1 mod 4⟩): controlled by coin in state |1⟩
            qc.x(2)
            self._increment_mod_4(qc, control=2, q0=0, q1=1)
            qc.x(2)

            # Left shift (|i-1 mod 4⟩): controlled by coin in state |0⟩
            self._decrement_mod_4(qc, control=2, q0=0, q1=1)

        # Measure position qubits
        qc.measure(0, 0)
        qc.measure(1, 1)

        # Run on simulator
        backend = AerSimulator()
        transpiled = transpile(qc, backend)
        result = backend.run(transpiled, shots=1).result()
        counts = result.get_counts()
        print(f"Quantum walk result counts: {counts}")
        measured = list(counts.keys())[0]
        pos = int(measured[::-1], 2)  # fix Qiskit's endianness
        print(f"Measured position: {pos}, corresponding to color: {color_map[pos]}")

        return color_map[pos]

    def _increment_mod_4(self, qc, control, q0, q1):
        """Controlled increment modulo 4 on 2 position qubits"""
        # q0 is LSB, q1 is MSB
        qc.cx(control, q0)
        qc.ccx(control, q0, q1)

    def _decrement_mod_4(self, qc, control, q0, q1):
        """Controlled decrement modulo 4 on 2 position qubits"""
        # Implemented as controlled subtract 1 mod 4
        # Truth table (q0, q1):
        # 00 -> 11
        # 01 -> 00
        # 10 -> 01
        # 11 -> 10

        # Inverse of increment
        qc.ccx(control, q0, q1)
        qc.cx(control, q0)

In [10]:
# quantum_draw_card.py
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator  # new simulator backend replacing BasicAer
from qiskit import transpile

class Quantum_draw_card(Card):
    def __init__(self, color, max_cards=8):
        super().__init__(color, f"Quantum Draw up to {max_cards}")
        self.max_cards = max_cards
        self.cardId = 13  # Unique identifier for this card type

    def play(self, game):
        """Play the quantum draw card effect."""
        drawn_cards = self.activate_quantum_effect()
        print(f"Number of cards to draw: {drawn_cards}")
        
        for _ in range(drawn_cards):
            card = game.deck.DrawCard()
            if card:
                game.get_next_player().Hand.append(card)
                print(f"Card drawn and added to next player's hand: {card}")

    def activate_quantum_effect(self):
        """Generate a quantum random number between 0 and max_cards (inclusive)."""
        n_qubits = 3  # Number of qubits to represent the range
        qc = QuantumCircuit(n_qubits, n_qubits)

        # Put all qubits into superposition
        for q in range(n_qubits):
            qc.h(q)

        qc.measure(range(n_qubits), range(n_qubits))
        sim = AerSimulator()
        qc = transpile(qc, sim)
        job = sim.run(qc)
        result = job.result()
        counts = result.get_counts()

        bitstring = list(counts.keys())[0]
        number = int(bitstring, 2)
        print(f"Measured number: {number}")

        # Rejection sampling if number > max_cards
        if number > self.max_cards:
            print(f"Number {number} exceeds max_cards {self.max_cards}, retrying...")
            return self.activate_quantum_effect()
        else:
            print(f"Returning valid number: {number}")
            return number

In [35]:
import random
from qiskit_aer import AerSimulator


class Quantum_grover_card(Card):
    def __init__(self, color="Purple"):
        super().__init__(color, "Quantum Grover")
        self.cardId = 17  # Just an arbitrary ID
        self.selected_card = None 

    def play(self, game, selected_card=None):
        current_player = game.get_current_player()
        other_players = [p for p in game.players if p != current_player]

        if self.selected_card is None:
            # Appel initial : l'UI doit déclencher la sélection
                choice = random.randrange(len(current_player.GetHand()))
                self.selected_card = current_player.GetHand()[choice]

        print(f"\n🔍 Grover scan for: {self.selected_card}\n")

        for player in other_players:
            Hand = player.GetHand()
            found = grover_card_search(Hand, self.selected_card, verbose=False)
            print(f"Grover search result for {player.GetName()}: {found}")
            if found:
                index = card_to_index(self.selected_card, Hand)
                print(f"✅ {player.GetName()} has {self.selected_card} at index {index}. Adding {index} card(s) as penalty.")
                for _ in range(index):
                    new_card = game.draw_card(game.players.index(player))
                    if new_card:
                        player.AddCard(new_card)
                        print(f"Card added to {player.GetName()}: {new_card}")
            else:
                print(f"❌ {player.GetName()} does not have {self.selected_card}.")

        game.discard_pile.append(self)
        print(f"Card {self} added to discard pile.")
        game.next_turn()
        print("Next turn triggered.")

In [13]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import math

class Quantum_shuffle_card(Card):
    """A card that shuffles all cards from all players using quantum-generated indices."""
    def __init__(self, color):
        super().__init__(color, "Quantum True Shuffle")
        self.cardId = 14

    def play(self, game):
        self.quantum_shuffle(game)

    def quantum_shuffle(self, game):
        # Collect all cards from players
        full_deck = []
        for player in game.players:
            full_deck.extend(player.Hand)
            player.Hand.clear()
        print(f"Collected full deck: {[str(card) for card in full_deck]}")

        num_players = len(game.players)
        player_index = 0

        while len(full_deck) > 0:
            # Determine number of qubits required to index current deck
            num_qubits = math.ceil(math.log2(len(full_deck)))
            if num_qubits == 0:
                idx = 0
            else:
                idx = self.quantum_random_index(num_qubits) % len(full_deck)
            print(f"Selected index: {idx} from full deck of size {len(full_deck)}")

            # Remove the card at the selected index and assign it to a player
            selected_card = full_deck.pop(idx)
            game.players[player_index].Hand.append(selected_card)
            print(f"Assigned card {selected_card} to player {player_index}")

            player_index = (player_index + 1) % num_players

        print("Shuffling complete. Player hands updated.")

    def quantum_random_index(self, n_qubits):
        """Generates a quantum random integer using n_qubits."""
        qc = QuantumCircuit(n_qubits, n_qubits)
        qc.h(range(n_qubits))
        qc.measure(range(n_qubits), range(n_qubits))

        sim = AerSimulator()
        transpiled = transpile(qc, sim)
        result = sim.run(transpiled, shots=1).result()
        counts = result.get_counts()
        print(f"Quantum random index counts: {counts}")

        bitstring = list(counts.keys())[0]
        number = int(bitstring, 2)
        print(f"Returning quantum random index: {number}")
        return number

In [25]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Quantum_swap_card(Card):
    """A card that swaps cards of two players using a quantum CSWAP gate, measuring the control and data qubits."""
    def __init__(self, color, max_cards=8):
        super().__init__(color, "Quantum Hand Swap")
        self.cardId = 16

    def play(self, game):
        self.activate_quantum_effect(game, game.get_current_player(), game.get_next_player())

    def activate_quantum_effect(self, game, player1, player2):
        n1, n2 = len(player1.Hand), len(player2.Hand)
        n_swap = min(n1, n2)

        if n_swap == 0:
            print("[Quantum SWAP] Nothing to swap. Returning.")
            return  # Nothing to swap

        # total qubits = 1 (control) + 2 * n_swap (cards)
        total_qubits = 1 + 2 * n_swap
        # Measure all qubits including control qubit
        qc = QuantumCircuit(total_qubits, total_qubits)

        # Control in superposition
        qc.h(0)

        # Apply CSWAP gates controlled by qubit 0
        for i in range(n_swap):
            qc.cswap(0, 1 + i, 1 + n_swap + i)

        # Measure all qubits
        qc.measure(range(total_qubits), range(total_qubits))

        # Run on simulator
        backend = AerSimulator()
        qc = transpile(qc, backend)
        result = backend.run(qc, shots=1).result()
        counts = result.get_counts()
        bitstring = list(counts.keys())[0][::-1]  # Reverse to align qubit index with bitstring index

        print(f"[Quantum SWAP] Measured bitstring: {bitstring}")

        control_bit = bitstring[0]

        new_hand1 = []
        new_hand2 = []

        if control_bit == '1':
            # Swap first n_swap cards
            for i in range(n_swap):
                new_hand1.append(player2.Hand[i])
                new_hand2.append(player1.Hand[i])
        else:
            # Keep hands as is for first n_swap cards
            for i in range(n_swap):
                new_hand1.append(player1.Hand[i])
                new_hand2.append(player2.Hand[i])

        # Append remaining cards (those beyond n_swap)
        new_hand1.extend(player1.Hand[n_swap:])
        new_hand2.extend(player2.Hand[n_swap:])

        # Apply new hands
        player1.Hand = new_hand1
        player2.Hand = new_hand2

        print(f"[Quantum SWAP] {player1.GetName()} new hand: {[str(c) for c in player1.Hand]}")
        print(f"[Quantum SWAP] {player2.GetName()} new hand: {[str(c) for c in player2.Hand]}")

In [26]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator  # or from qiskit.providers.aer import AerSimulator

import random

class teleportation_card(Card):
    def __init__(self, color):
        super().__init__(color, "Teleportation")
        self.cardId = 18  # Unique identifier for this card type

    def play(self, game):
        # Run the teleportation circuit
        measurement = self.activate_teleportation()
        print(f"[Teleportation Card] Measurement result: {measurement}")

        # Interpret measurement outcome (bitstring like '00', '01', '10', or '11')
        ones_count = measurement.count('1')
        print(f"[Teleportation Card] Number of '1's in measurement: {ones_count}")

        # For example, swap cards between current and next player equal to number of 1s (max 2)
        num_swaps = ones_count

        if num_swaps == 0:
            print("[Teleportation Card] No swaps to perform. Returning.")
            return

        curr_player = game.get_current_player()
        next_player = game.get_next_player()

        swaps_done = []

        for _ in range(num_swaps):
            if not curr_player.Hand or not next_player.Hand:
                print("[Teleportation Card] One of the players has no cards left to swap. Breaking.")
                break

            # Pick random cards to swap
            card_from_curr = random.choice(curr_player.Hand)
            card_from_next = random.choice(next_player.Hand)

            curr_player.Hand.remove(card_from_curr)
            next_player.Hand.remove(card_from_next)

            curr_player.Hand.append(card_from_next)
            next_player.Hand.append(card_from_curr)

            swaps_done.append((str(card_from_curr), str(card_from_next)))

        msg = f"Teleportation swapped {len(swaps_done)} cards between {curr_player.Name} and {next_player.Name}:\n"
        msg += "\n".join([f"{curr} <--> {nxt}" for curr, nxt in swaps_done])
        print(msg)

    def activate_teleportation(self):
        qc = QuantumCircuit(3, 2)

        # Prepare |+> state to teleport (can be changed)
        qc.h(0)

        # Create Bell pair between qubit 1 and 2
        qc.h(1)
        qc.cx(1, 2)

        # Bell measurement on qubit 0 and 1
        qc.cx(0, 1)
        qc.h(0)

        qc.measure([0, 1], [0, 1])

        sim = AerSimulator()
        qc = transpile(qc, sim)
        job = sim.run(qc)
        result = job.result()
        counts = result.get_counts()

        # Return the measurement bitstring (e.g. '00', '01', '10', or '11')
        measurement = list(counts.keys())[0]
        print(f"[Teleportation Card] Quantum circuit measurement: {measurement}")
        return measurement

In [27]:
import random
def create_card_by_id(card_id):
    """Return an instance of a card based on its ID."""

    colors = ["Red", "Green", "Blue", "Yellow"]

    # Importer localement ici pour éviter les circular imports
    if card_id == 15:
        from uno.cards.Quantum_superposed_card import Quantum_superposed_card
        return Quantum_superposed_card(random.choice(colors), 8)

    card_map = {
        0: Card(random.choice(colors), 0),
        1: Card(random.choice(colors), 1),
        2: Card(random.choice(colors), 2),
        3: Card(random.choice(colors), 3),
        4: Card(random.choice(colors), 4),
        5: Card(random.choice(colors), 5),
        6: Card(random.choice(colors), 6),
        7: Card(random.choice(colors), 7),
        8: Card(random.choice(colors), 8),
        9: Card(random.choice(colors), 9),
        10: quantum_balance_card(random.choice(colors)),
        11: quantum_card(random.choice(colors), random.randint(0, 9)),
        12: Quantum_color_card(colors, random.choice(colors)),
        13: Quantum_draw_card(random.choice(colors), 8),
        14: Quantum_shuffle_card(random.choice(colors)),
        16: Quantum_swap_card(random.choice(colors)),
        17: Quantum_grover_card(random.choice(colors)),
        18: teleportation_card(random.choice(colors)),
    }

    return card_map.get(card_id, None)


In [18]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Quantum_superposed_card(Card):
    """A card that creates a superposition over all possible card IDs and adds one to the next player."""
    def __init__(self, color, max_card_id=18):
        super().__init__(color, "Quantum Superposed Card")
        self.max_card_id = max_card_id
        self.cardId = 15

    def play(self, game):
        """Generate a card ID via quantum effect and add the corresponding card to the next player's hand."""
        card_id = self.activate_quantum_effect()
        print(f"[Quantum Superposed Card] Generated card ID: {card_id}")

        # Create the corresponding card
        card = create_card_by_id(card_id)

        if card:
            print(f"[Quantum Superposed Card] Added card with ID {card_id} to next player.")
            game.get_next_player().Hand.append(card)
        else:
            print(f"[Quantum Superposed Card] Invalid card ID generated: {card_id}")

    def activate_quantum_effect(self):
        """Quantum generation of an integer between 0 and max_card_id inclusive."""
        n_qubits = 5  # 2^5 = 32 > max_card_id
        qc = QuantumCircuit(n_qubits, n_qubits)

        # Put all qubits into superposition
        for q in range(n_qubits):
            qc.h(q)

        qc.measure(range(n_qubits), range(n_qubits))

        sim = AerSimulator()
        qc = transpile(qc, sim)
        result = sim.run(qc, shots=1).result()
        counts = result.get_counts()
        print(f"[Quantum Superposed Card] Quantum circuit result counts: {counts}")

        bitstring = list(counts.keys())[0]
        number = int(bitstring, 2) % (self.max_card_id + 1)
        print(f"[Quantum Superposed Card] Measured bitstring: {bitstring} -> cardId {number}")

        print(f"[Quantum Superposed Card] Returning card ID: {number}")
        return number

In [21]:
class Deck:

    def __init__(self):
        """Initialize the deck with an empty pile and discard list."""
        self.CardInPile = []
        self.CardDiscarted = []
        self.AllTheCardType = []
    def PlayCard(self, card):
        # move a played card to discard pile
        self.CardDiscarted.append(card)
    def AddCard(self, card):
        """Add a card to the deck."""
        self.CardInPile.append(card)
    def DrawCard(self):
        # draw top card from pile
        return self.CardInPile.pop(0) if self.CardInPile else None
    def GetPileSize(self):
        """Return the number of cards in the pile."""
        return len(self.CardInPile)

In [22]:
class Player:
    def __init__(self, name):
        """Initialize the player with a name, an empty Hand, and an empty list of played cards."""
        self.Name = name
        self.Hand = []
        self.CardPlayed = []
        self.TurnNumber = None  # initialize turn number

    def AddCard(self, card):
        """Add a card to the player's Hand."""
        self.Hand.append(card)

    def PlayCard(self, card_index):
        """Play a card from the player's Hand by index."""
        if 0 <= card_index < len(self.Hand):
            card = self.Hand.pop(card_index)
            self.CardPlayed.append(card)
            return card
        else:
            raise IndexError("Invalid card index.")

    def DrawCard(self, card):
        """Add a card to the player's Hand."""
        self.Hand.append(card)

    def GetHand(self):
        """Return the player's Hand."""
        return self.Hand

    def GetName(self):
        """Return the player's name."""
        return self.Name

    def GetCardPlayed(self):
        """Return the cards played by the player."""
        return self.CardPlayed
    
    def GetCard(sef, cardIndex):
        return self.Hand[cardIndex] if 0 <= cardIndex < len(self.Hand) else None
    
    def is_bot(self):
        """Check if the player is a bot."""
        return self.Name=="Bot" or self.Name=="BotCode"

In [ ]:
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

class Game:
    def __init__(self):
        """Initialize the game with an empty player list, turn list, and card in play."""
        self.deck = Deck()
        self.discard_pile = []
        self.players = []
        self.current_player_idx = 0
    def add_player(self, name):
        """Add a player by name."""
        self.players.append(Player(name))

    def build_deck(self):
        """Populate and shuffle the deck."""
        colors = ["Red", "Green", "Blue", "Yellow"]
        values = [str(n) for n in range(0, 10)]
        # Add number cards
        for color in colors:
            self.deck.CardInPile.append(Card(color, "0"))
            for v in values[1:]:
                self.deck.CardInPile.append(Card(color, v))
                self.deck.CardInPile.append(Card(color, v))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum"))
        self.deck.CardInPile.append(Quantum_draw_card("Red", 8))
        self.deck.CardInPile.append(Quantum_draw_card("Blue", 8))
        self.deck.CardInPile.append(Quantum_draw_card("Green", 8))
        self.deck.CardInPile.append(Quantum_draw_card("Yellow", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Red", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Blue", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Green", 8))
        self.deck.CardInPile.append(Quantum_superposed_card("Yellow", 8))
        self.deck.CardInPile.append(Quantum_swap_card("Red"))
        self.deck.CardInPile.append(Quantum_swap_card("Blue"))
        self.deck.CardInPile.append(Quantum_swap_card("Green"))
        self.deck.CardInPile.append(Quantum_swap_card("Yellow"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Red"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Blue"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Green"))
        self.deck.CardInPile.append(Quantum_shuffle_card("Yellow"))
        # self.deck.CardInPile.append(quantum_balance_card("Red"))
        # self.deck.CardInPile.append(quantum_balance_card("Blue"))
        # self.deck.CardInPile.append(quantum_balance_card("Green"))
        # self.deck.CardInPile.append(quantum_balance_card("Yellow"))
        self.deck.CardInPile.append(quantum_card("Red", 8))
        self.deck.CardInPile.append(quantum_card("Blue", 8))
        self.deck.CardInPile.append(quantum_card("Green", 8))
        self.deck.CardInPile.append(quantum_card("Yellow", 8))
        self.deck.CardInPile.append(Quantum_grover_card("Red"))
        self.deck.CardInPile.append(Quantum_grover_card("Blue"))
        self.deck.CardInPile.append(Quantum_grover_card("Green"))
        self.deck.CardInPile.append(Quantum_grover_card("Yellow"))
        self.deck.CardInPile.append(teleportation_card("Red"))
        self.deck.CardInPile.append(teleportation_card("Blue"))
        self.deck.CardInPile.append(teleportation_card("Green"))
        self.deck.CardInPile.append(teleportation_card("Yellow"))

        #self.QuantumShuffleDeck()
        random.shuffle(self.deck.CardInPile)

    def deal(self):
        """Deal 7 cards to each player"""
        for _ in range(7):
            for player in self.players:
                if self.deck.CardInPile:
                    player.AddCard(self.deck.CardInPile.pop(0))

    def start(self):
        """Initial setup: build deck, deal, and flip first card."""
        self.build_deck()
        self.deal()
        # flip first card
        if self.deck.CardInPile:
            first = self.deck.CardInPile.pop(0)
            self.discard_pile.append(first)

    def get_current_player(self):
        return self.players[self.current_player_idx]
    
    def get_next_player(self):
        return self.players[(self.current_player_idx + 1) % len(self.players)]

    def get_top_card(self):
        return self.discard_pile[-1] if self.discard_pile else None

    def draw_card(self, player_idx):
        """Controller handles drawing a card."""
        if self.deck.CardInPile:
            card = self.deck.CardInPile.pop(0)
            self.players[player_idx].AddCard(card)
            return card
        return None

    def play_card(self, player_idx, card_idx):
        """Controller handles playing a card."""
        played = self.players[player_idx].PlayCard(card_idx)
        played.play(self)
        self.discard_pile.append(played)
        return played

    def next_turn(self):
        self.current_player_idx = (self.current_player_idx + 1) % len(self.players)

    def quantum_detect_winner_index(self):
        """
        Simule un circuit quantique où l'index du joueur avec main vide est encodé en binaire.
        Suppose qu'un seul joueur a une main vide.
        """
        num_players = len(self.players)
        num_qubits = (num_players - 1).bit_length()

        # Cherche index du joueur gagnant
        winner_index = None
        for i, player in enumerate(self.players):
            if not player.GetHand():
                winner_index = i
                break

        if winner_index is None:
            return None  # Aucun gagnant

        # Encode l’index dans un registre quantique
        qc = QuantumCircuit(num_qubits, num_qubits)
        bin_index = format(winner_index, f'0{num_qubits}b')

        for i, bit in enumerate(reversed(bin_index)):
            if bit == '1':
                qc.x(i)

        qc.measure(range(num_qubits), range(num_qubits))

        sim = AerSimulator()
        qc = transpile(qc, sim)
        result = sim.run(qc, shots=1).result()
        counts = result.get_counts()

        measured = list(counts.keys())[0]
        return int(measured, 2)  # Reverse for Qiskit's LSB convention

    def has_winner(self):
        """Return winning player or None."""
        idx = self.quantum_detect_winner_index()
        if idx is not None:
            return self.players[idx]
        return None
    
    def QuantumShuffleDeck(self):
        """Shuffle the deck using quantum principles."""
        shuffled_deck = []
        """Generate a quantum random number between 0 and max_cards (inclusive)."""
        n_qubits = 3  #
        qc = QuantumCircuit(n_qubits, n_qubits)

        # Put all qubits into superposition
        for q in range(n_qubits):
            qc.h(q)

        qc.measure(range(n_qubits), range(n_qubits))
        sim = AerSimulator()    
        qc = transpile(qc, sim)
        job = sim.run(qc)
        result = job.result()
        counts = result.get_counts()

        bitstring = list(counts.keys())[0]
        number = int(bitstring, 2)
        
        while self.deck:
            shuffled_deck.append(self.deck.CardInPile.pop(number% len(self.deck.CardInPile)))
        self.deck = shuffled_deck

    def reverse_turn_order(self):
        """Reverse the turn order of players."""
        self.players.reverse()
        self.current_player_idx = len(self.players) - 1 - self.current_player_idx

    def get_card_by_idx(self, card_idx):
        player = self.get_current_player()
        if 0 <= card_idx < len(player.Hand):
            return player.Hand[card_idx]
        return None
        

In [24]:
Game=Game()
Game.add_player("Player 1")
Game.add_player("Player 2")
Game.start()

Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 


In [30]:
CardEffect=quantum_balance_card(Game).play(Game)

Card weight for Red 8: 5
Card weight for Yellow 7: 5
Card weight for Green Quantum True Shuffle: 5
Card weight for Blue Quantum Superposed Card: 5
Card weight for Red Quantum Superposed Card: 5
Card weight for Red 2: 5
Card weight for Yellow 6: 5
Card weight for Blue 3: 5
Card weight for Blue Quantum Draw up to 8: 5
Card weight for Green 7: 5
Card weight for Red Quantum True Shuffle: 5
Card weight for Green 0: 5
Card weight for Blue 1: 5
Card weight for Yellow 3: 5
Cost function value: 1325.9619140625
Cost function value: 783.189697265625
Cost function value: 819.228515625
Cost function value: 941.45263671875
Cost function value: 1606.4013671875
Cost function value: 674.676513671875
Cost function value: 376.162109375
Cost function value: 412.327880859375
Cost function value: 422.21435546875
Cost function value: 575.113525390625
Cost function value: 1511.34521484375
Cost function value: 932.613525390625
Cost function value: 600.0439453125
Cost function value: 1408.941650390625
Cost func

In [31]:
CardEffect=quantum_card("Red", 8).play(Game)

Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Quantum effect result counts: {'1': 1}
Counts from quantum effect: {'1': 1}
Effect: Reversing turn order.


In [32]:
CardEffect=Quantum_color_card(["Red", "Blue", "Green", "Yellow"],"Quantum").play(Game)

Current color set to: Red
Initial position index: 0
Quantum walk result counts: {'11': 1}
Measured position: 3, corresponding to color: Yellow
New color chosen by quantum walk: Yellow


In [33]:
CardEffect=Quantum_draw_card("Red", 8).play(Game)

Measured number: 6
Returning valid number: 6
Number of cards to draw: 6
Card drawn and added to next player's hand: Yellow 4
Card drawn and added to next player's hand: Yellow 9
Card drawn and added to next player's hand: Green 2
Card drawn and added to next player's hand: Red 8
Card drawn and added to next player's hand: Green Quantum Hand Swap
Card drawn and added to next player's hand: Green 9


In [36]:
CardEffect=Quantum_grover_card("Red").play(Game)


🔍 Grover scan for: Red 2

Checking card at index 0: Red 8
Checking card at index 1: Yellow 7
Checking card at index 2: Red Quantum Superposed Card
Checking card at index 3: Blue 3
Checking card at index 4: Blue Quantum Draw up to 8
Checking card at index 5: Red Quantum True Shuffle
Checking card at index 6: Yellow 3
Checking card at index 7: Yellow 4
Checking card at index 8: Yellow 9
Checking card at index 9: Green 2
Checking card at index 10: Red 8
Checking card at index 11: Green Quantum Hand Swap
Checking card at index 12: Green 9
Returning index: -1
Returning False: card not found in hand.
Grover search result for Player 2: False
❌ Player 2 does not have Red 2.
Card Red Quantum Grover added to discard pile.
Next turn triggered.


In [37]:
CardEffect=Quantum_shuffle_card("Red").play(Game)

Collected full deck: ['Red 8', 'Yellow 7', 'Red Quantum Superposed Card', 'Blue 3', 'Blue Quantum Draw up to 8', 'Red Quantum True Shuffle', 'Yellow 3', 'Yellow 4', 'Yellow 9', 'Green 2', 'Red 8', 'Green Quantum Hand Swap', 'Green 9', 'Green Quantum True Shuffle', 'Blue Quantum Superposed Card', 'Red 2', 'Yellow 6', 'Green 7', 'Green 0', 'Blue 1']
Quantum random index counts: {'11101': 1}
Returning quantum random index: 29
Selected index: 9 from full deck of size 20
Assigned card Green 2 to player 0
Quantum random index counts: {'11011': 1}
Returning quantum random index: 27
Selected index: 8 from full deck of size 19
Assigned card Yellow 9 to player 1
Quantum random index counts: {'01010': 1}
Returning quantum random index: 10
Selected index: 10 from full deck of size 18
Assigned card Green 9 to player 0
Quantum random index counts: {'00111': 1}
Returning quantum random index: 7
Selected index: 7 from full deck of size 17
Assigned card Yellow 4 to player 1
Quantum random index counts:

In [38]:
CardEffect=Quantum_superposed_card("Red", 8).play(Game)

[Quantum Superposed Card] Quantum circuit result counts: {'10100': 1}
[Quantum Superposed Card] Measured bitstring: 10100 -> cardId 2
[Quantum Superposed Card] Returning card ID: 2
[Quantum Superposed Card] Generated card ID: 2
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
[Quantum Superposed Card] Added card with ID 2 to next player.


In [39]:
CardEffect=Quantum_swap_card("Red").play(Game)

[Quantum SWAP] Measured bitstring: 100000000000000000000
[Quantum SWAP] Player 2 new hand: ['Yellow 9', 'Yellow 4', 'Blue Quantum Superposed Card', 'Red 8', 'Red Quantum True Shuffle', 'Red 2', 'Red Quantum Superposed Card', 'Green 7', 'Blue 3', 'Green 0']
[Quantum SWAP] Player 1 new hand: ['Green 2', 'Green 9', 'Red 8', 'Blue Quantum Draw up to 8', 'Green Quantum True Shuffle', 'Green Quantum Hand Swap', 'Yellow 6', 'Yellow 7', 'Blue 1', 'Yellow 3', 'Red 2']


In [40]:
CardEffect=teleportation_card("Red").play(Game)

[Teleportation Card] Quantum circuit measurement: 01
[Teleportation Card] Measurement result: 01
[Teleportation Card] Number of '1's in measurement: 1
Teleportation swapped 1 cards between Player 2 and Player 1:
Yellow 9 <--> Green Quantum Hand Swap


In [41]:
Game.quantum_detect_winner_index()

In [43]:
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
from qiskit_ibm_runtime import EstimatorV2 as Estimator, SamplerV2 as Sampler
from qiskit_aer import AerSimulator
from qiskit import transpile
from scipy.optimize import minimize
import numpy as np
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_optimization.translators import to_ising

class BotCode(Player):
    def __init__(self, name="Bot"):
        super().__init__(name)
        self.is_bot = True

    def card_weight(self, card):
        if card.value == 'Draw Two':
            return 20
        elif card.value in ['Skip', 'Reverse']:
            return 10
        elif isinstance(card.value, int):
            return 1
        else:
            return 5

    def is_playable(self, card, top_card):
        return card.color == top_card.color or card.value == top_card.value

    def generate_qubo(self, Hand, top_card):
        mdl = Model("BotDecision")
        m = len(Hand)
        x = [mdl.binary_var(name=f"x_{i}") for i in range(m)]

        print(f"Hand: {[str(card) if self.is_playable(card, top_card) else 0 for card in Hand]}")
        weights = [self.card_weight(card) if self.is_playable(card, top_card) else 0 for card in Hand]

        obj = mdl.sum(-weights[i] * x[i] for i in range(m))
        constraint = mdl.sum(x) - 1
        alpha = 100
        mdl.minimize(obj + alpha * constraint * constraint)

        qp = from_docplex_mp(mdl)
        qubo = QuadraticProgramToQubo().convert(qp)
        print(f"Returning QUBO: {qubo}, Weights: {weights}")
        return qubo, weights

    def cost_function(self, params, estimator, circuit, hamiltonian, pass_manager):
        isa_psi = pass_manager.run(circuit)
        isa_observables = hamiltonian.apply_layout(isa_psi.layout)
        job = estimator.run([(isa_psi, isa_observables, params)])
        cost = job.result()[0].data.evs
        print(f"Returning cost: {cost}")
        return cost

    def decide_action(self, game):
        top_card = game.get_top_card()
        Hand = self.GetHand()

        if not Hand:
            print("Returning action: ('draw', None)")
            return ('draw', None)

        qubo, weights = self.generate_qubo(Hand, top_card)
        hamiltonian, _ = to_ising(qubo)

        backend = AerSimulator()
        estimator = Estimator(mode=backend)
        pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=3)

        circuit_qaoa = QAOAAnsatz(hamiltonian, reps=10)
        p = circuit_qaoa.num_parameters // 2
        gamma_init = np.linspace(0.1, 1.5, p)
        beta_init = np.linspace(0.1, 1.5, p)
        params_init = np.concatenate([gamma_init, beta_init])

        res_opt = minimize(
            self.cost_function,
            params_init,
            args=(estimator, circuit_qaoa, hamiltonian, pass_manager),
            method="COBYLA"
        )

        params_opt = res_opt.x
        sampler = Sampler(mode=backend)

        circuit_meas = circuit_qaoa.decompose(reps=2).copy()
        circuit_meas.measure_all()

        counts = sampler.run([(circuit_meas, params_opt)]).result()[0].data.meas.get_counts()
        if isinstance(counts, dict):
            most_likely_bitstring = max(counts, key=counts.get)
        else:
            most_likely_bitstring = next(iter(counts), None)
            
        sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
        most_likely_bitstring = sorted_counts[0][0] if len(sorted_counts) > 0 else None
        second_most_likely_bitstring = sorted_counts[1][0] if len(sorted_counts) > 1 else None
        if most_likely_bitstring == "0" * len(most_likely_bitstring):
            print("Most likely bitstring is all zeros, using second most likely.")
            most_likely_bitstring = second_most_likely_bitstring

        print(f"Most likely bitstring: {most_likely_bitstring}")

        # Indices jouables
        playable_indices = [i for i, card in enumerate(Hand) if self.is_playable(card, top_card)]

        if most_likely_bitstring is not None:
            selected_index = None
            max_weight = -1
            for i in range(len(playable_indices)):
                bit_val = int(most_likely_bitstring[::-1][i])
                real_index = playable_indices[i]
                print(f"Card {real_index}: {Hand[real_index]}, Bit: {bit_val}")
                if bit_val == 1:
                    print(f"Returning action: ('play', {real_index})")
                    return ('play', real_index)

            print("Bitstring has no 1s, fallback triggered.")
            for i in playable_indices:
                w = self.card_weight(Hand[i])
                if w > max_weight:
                    max_weight = w
                    selected_index = i

            if selected_index is not None:
                print(f"Fallback: manually selecting card {selected_index} -> {Hand[selected_index]}")
                print(f"Returning action: ('play', {selected_index})")
                return ('play', selected_index)

        print("No valid move, drawing a card.")
        print("Returning action: ('draw', None)")
        return ('draw', None)

    def take_turn(self, game):
        idx = game.players.index(self)
        action, card_idx = self.decide_action(game)
        if action == 'play' and card_idx is not None:
            played = game.play_card(idx, card_idx)
            print(f"Returning action result: ('play', {played})")
            return ('play', played)
        drawn = game.draw_card(idx)
        print(f"Returning action result: ('draw', {drawn})")
        return ('draw', drawn)

In [ ]:

def test_botcode():
    # Initialize the game
    Game.add_player("Player 1")
    bot = BotCode(name="Bot")
    Game.players.append(bot)
    Game.start()

    # Add some cards to the bot's hand
    bot.Hand = [
        Card("Red", "5"),
        Card("Blue", "Draw Two"),
        Card("Green", "Reverse"),
        Card("Yellow", "Skip"),
    ]

    # Set the top card on the discard pile
    Game.discard_pile.append(Card("Red", "7"))

    # Test card_weight
    print("\nTesting card_weight:")
    for card in bot.Hand:
        weight = bot.card_weight(card)
        print(f"Card: {card}, Weight: {weight}")

    # Test is_playable
    print("\nTesting is_playable:")
    top_card = Game.get_top_card()
    for card in bot.Hand:
        playable = bot.is_playable(card, top_card)
        print(f"Card: {card}, Top Card: {top_card}, Playable: {playable}")

    # Test generate_qubo
    print("\nTesting generate_qubo:")
    qubo, weights = bot.generate_qubo(bot.Hand, top_card)
    print(f"Generated QUBO: {qubo}")
    print(f"Weights: {weights}")

    # Test decide_action
    print("\nTesting decide_action:")
    action, card_idx = bot.decide_action(Game)
    print(f"Decided Action: {action}, Card Index: {card_idx}")
    if action == "play" and card_idx is not None:
        print(f"Card to Play: {bot.Hand[card_idx]}")

    # Test take_turn
    print("\nTesting take_turn:")
    result = bot.take_turn(Game)
    print(f"Result of Bot's Turn: {result}")

test_botcode()

Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 
Created quantum circuit:         ┌───┐ ░ ┌─┐
     q: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0 

Testing card_weight:
Card: Red 5, Weight: 5
Card: Blue Draw Two, Weight: 20
Card: Green Reverse, Weight: 10
Card: Yellow Skip, Weight: 10

Testing is_playable:
Card: Red 5, Top Card: Red 7, Playable: True
Card: Blue Draw Two, Top Card: Red 7, Playable: False
Card: Green Reverse, Top Card: Red 7, Playable: False
Card: Yellow Skip, Top Card: Red 7, Playable: False

Testing generate_qubo:
Hand: ['Red 5', 0, 0, 0]
Returning QUBO: minimize 100*x_0^2 + 200*x_0*x_1 + 200*x_0*x_2 + 200*x_0*x_3 + 100*x_

NameError: name 'game' is not defined